# データを整形，特徴量作成を行うスクリプトを実装

In [2]:

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

plt.style.use('dark_background')
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
items = pd.read_csv("../data/input/items.csv")
item_categories = pd.read_csv("../data/input/item_categories.csv")
shops = pd.read_csv("../data/input/shops.csv")
train = pd.read_csv("../data/input/sales_train.csv.gz")
test = pd.read_csv("../data/input/test.csv.gz")
sample_predict = pd.read_csv("../data/input/sample_submission.csv.gz")

In [4]:
train_s = pd.merge(train, shops, on="shop_id", sort=False)
train_si = pd.merge(train_s, items, on="item_id", sort=False)
train_siic = pd.merge(train_si, item_categories, on="item_category_id", sort=False)

In [12]:
# 外れ値を除く, 現状は販売個数上位１位のレコードのみ

def remove_outlier(df_input):
    df = df_input.copy()
    df = df.drop(index=df.item_cnt_day.argmax()).reset_index()
    return df

In [60]:
# shop_id に対する特徴量を作成, 標準化等は最後のステップとする？？
def encode_shop_id(df):
    
    df = train.copy()
    shop_ids = pd.DataFrame(df['shop_id']).drop_duplicates().sort_values('shop_id').reset_index(drop=True)
    
    ## 月間売上個数
    monthly_sale = df.groupby(['date_block_num', 'shop_id']).sum()['item_cnt_day'].reset_index()
    
    ## 総売上個数, ランク作成 (rank は大きいほど販売個数が多いことを示す)
    total_sale = monthly_sale.groupby('shop_id').sum()['item_cnt_day'].reset_index().rename(columns={'item_cnt_day': 'item_cnt_total'})
    total_sale['item_cnt_total_rank'] = total_sale.rank()['item_cnt_total']
    
    ## 月間平均売上個数, ランク作成
    average_monthly_sale = monthly_sale.groupby('shop_id').mean()['item_cnt_day'].reset_index().rename(columns={'item_cnt_day': 'item_cnt_monthly_average'})
    average_monthly_sale['item_cnt_monthly_average_rank'] = average_monthly_sale.rank()['item_cnt_monthly_average']
    
    ## 売ることができたアイテムの平均価格？？ -> そのお店で特定の値段レンジのアイテムが販売されたときに売れるかどうかの情報になるかも, レンジ化等は後段の処理とする？？
    df['total_price'] = df['item_price'] * df['item_cnt_day']
    average_sold_price = (df.groupby('shop_id').sum()['total_price'] / df.groupby('shop_id').sum()['item_cnt_day'])
    
    .reset_index().rename(columns={'item_price': 'item_price_average'})
    
    shop_feature = pd.merge(shop_ids, total_sale, on='shop_id', how='left')
    shop_feature = pd.merge(shop_feature, average_monthly_sale, on='shop_id', how='left')
    shop_feature = pd.merge(shop_feature, average_sold_price, on='shop_id', how='left')

    
    return shop_feature
    

In [61]:
# item_id に対する特徴量を作成, 標準化等は最後のステップとする？？
def encode_item_id(df):
    
    df = train.copy()
    item_ids = pd.DataFrame(df['item_id']).drop_duplicates().sort_values('item_id').reset_index(drop=True)
    
    ## 月間売上個数
    monthly_sale = df.groupby(['date_block_num', 'item_id']).sum()['item_cnt_day'].reset_index()
    
    ## 総売上個数, ランク作成 (rank は大きいほど販売個数が多いことを示す)
    total_sale = monthly_sale.groupby('item_id').sum()['item_cnt_day'].reset_index().rename(columns={'item_cnt_day': 'item_cnt_total'})
    total_sale['item_cnt_total_rank'] = total_sale.rank()['item_cnt_total']

    
    ## 月間平均売上個数, ランク作成
    average_monthly_sale = monthly_sale.groupby('item_id').mean()['item_cnt_day'].reset_index().rename(columns={'item_cnt_day': 'item_cnt_monthly_average'})
    average_monthly_sale['item_cnt_monthly_average_rank'] = average_monthly_sale.rank()['item_cnt_monthly_average']
    
    ## 平均販売価格を作成
    average_price = df.groupby('item_id').mean()['item_price'].reset_index().rename(columns={'item_price': 'item_price_average'})
    
    item_feature = pd.merge(item_ids, total_sale, on='item_id', how='left')
    item_feature = pd.merge(item_feature, average_monthly_sale, on='item_id', how='left')
    item_feature = pd.merge(item_feature, average_price, on='item_id', how='left')
    
    return item_feature

In [64]:

    df['total_price'] = df['item_price'] * df['item_cnt_day']
    average_sold_price = (df.groupby('shop_id').sum()['total_price'] / df.groupby('shop_id').sum()['item_cnt_day'])

In [66]:

average_sold_price.reset_index()

,shop_id,0
0,0,567.054250
1,1,513.105213
2,2,1438.590425
3,3,1062.981974
4,4,922.571806
5,5,892.391119
6,6,975.902375
7,7,998.503662
8,8,653.507093
9,9,1031.944851


In [63]:
encode_item_id(train)
encode_shop_id(train)

,item_id,item_cnt_total,item_cnt_total_rank,item_cnt_monthly_average,item_cnt_monthly_average_rank,item_price_average
0,0,1.0,1168.0,1.000000,1961.5,58.000000
1,1,6.0,5230.5,1.200000,4224.5,4490.000000
2,2,2.0,2841.0,1.000000,1961.5,58.000000
3,3,2.0,2841.0,1.000000,1961.5,79.000000
4,4,1.0,1168.0,1.000000,1961.5,58.000000
5,5,1.0,1168.0,1.000000,1961.5,28.000000
6,6,1.0,1168.0,1.000000,1961.5,100.000000
7,7,1.0,1168.0,1.000000,1961.5,28.000000
8,8,2.0,2841.0,1.000000,1961.5,58.000000
9,9,1.0,1168.0,1.000000,1961.5,58.000000


,shop_id,item_cnt_total,item_cnt_total_rank,item_cnt_monthly_average,item_cnt_monthly_average_rank,item_price_average
0,0,11705.0,11.0,5852.500000,57.0,563.444151
1,1,6311.0,7.0,3155.500000,51.0,515.350652
2,2,30620.0,20.0,900.588235,11.0,1350.638391
3,3,28355.0,19.0,833.970588,9.0,1036.840634
4,4,43942.0,23.0,1292.411765,16.0,923.994318
5,5,42762.0,22.0,1295.818182,17.0,871.182496
6,6,100489.0,53.0,2955.558824,50.0,901.955104
7,7,67058.0,41.0,1972.294118,33.0,965.581388
8,8,3595.0,3.0,898.750000,10.0,652.483208
9,9,15866.0,12.0,3966.500000,52.0,1256.812248
